In [8]:
from utils import (
    get_areas, 
    get_specimen_ids, 
    get_trials,
    get_units_count,
    get_summary_df,
    load_index
)

from neural_maps import (
    sim_corrected_source_pair, 
    pls_corrected_single_source_to_B
)

from mapping import (
    consistency_across_trials,
    interanimal_consistency_1v1,
    interanimal_consistency_pool
)

In [9]:
index_df = load_index("../PreprocData/PreprocData/index.csv")
index_df

,area,area_code,specimen_id,n_units,store,meta
0,AL,VISal,717038288,85,PreprocData/AL_717038288.npy,PreprocData/AL_717038288.json
1,AL,VISal,738651054,127,PreprocData/AL_738651054.npy,PreprocData/AL_738651054.json
2,AL,VISal,742714475,184,PreprocData/AL_742714475.npy,PreprocData/AL_742714475.json
3,AL,VISal,745276236,91,PreprocData/AL_745276236.npy,PreprocData/AL_745276236.json
4,AL,VISal,757329624,166,PreprocData/AL_757329624.npy,PreprocData/AL_757329624.json
5,AL,VISal,775876828,89,PreprocData/AL_775876828.npy,PreprocData/AL_775876828.json
6,AM,VISam,730756780,70,PreprocData/AM_730756780.npy,PreprocData/AM_730756780.json
7,AM,VISam,730760270,94,PreprocData/AM_730760270.npy,PreprocData/AM_730760270.json
8,AM,VISam,732548380,71,PreprocData/AM_732548380.npy,PreprocData/AM_732548380.json
9,AM,VISam,734865738,72,PreprocData/AM_734865738.npy,PreprocData/AM_734865738.json


In [10]:
summary_data_df = get_summary_df(index_df)
summary_data_df

,Area,Number of Specimen IDs,Total Units,Units per Specimen ID
0,AL,6,742,"[85, 127, 184, 91, 166, 89]"
1,AM,7,586,"[70, 94, 71, 72, 70, 74, 135]"
2,LM,6,369,"[51, 53, 74, 56, 77, 58]"
3,PM,5,386,"[65, 115, 62, 90, 54]"
4,RL,7,565,"[76, 69, 67, 68, 79, 95, 111]"
5,V1,8,748,"[110, 102, 91, 85, 88, 93, 94, 85]"


## Comparing a mouse with itself across different areas

Fixed specimen id, different trials, for each area.

In [ ]:
consistency_rsa = consistency_across_trials(index_df, sim_metric='RSA')
consistency_cka = consistency_across_trials(index_df, sim_metric='CKA')
consistency_cka

In [ ]:
consistency_PLS = consistency_across_trials(index_df, sim_metric='PLS')
consistency_PLS

# 1 vs 1 mapping: interanimal consistency

In [11]:
interanimal_consistency_1v1_df_rsa = interanimal_consistency_1v1(index_df, sim_metric='RSA')

interanimal_consistency_1v1_df_rsa.groupby('Area')['Mean'].median()

Area
AL    0.525163
AM    0.443280
LM    0.298855
PM    0.388201
RL    0.378969
V1    0.432187
Name: Mean, dtype: float64

In [12]:
interanimal_consistency_1v1_df_cka = interanimal_consistency_1v1(index_df, sim_metric='CKA')

interanimal_consistency_1v1_df_cka.groupby('Area')['Mean'].median()

Area
AL    0.447451
AM    0.438453
LM    0.207874
PM    0.344874
RL    0.299902
V1    0.410353
Name: Mean, dtype: float64

## Pooling version

In [13]:
iac_pool_rsa = interanimal_consistency_pool(index_df, sim_metric='RSA')
iac_pool_rsa.groupby('Area')['Mean'].median()

Area
AL    0.651665
AM    0.604958
LM    0.475808
PM    0.580446
RL         NaN
V1    0.624521
Name: Mean, dtype: float64

In [14]:
iac_pool_cka = interanimal_consistency_pool(index_df, sim_metric='CKA')
iac_pool_cka.groupby('Area')['Mean'].median()

Area
AL    0.558679
AM    0.600113
LM    0.394922
PM    0.510371
RL         NaN
V1    0.648063
Name: Mean, dtype: float64

In [ ]:
iac_pool_pls = interanimal_consistency_pool(index_df, sim_metric='PLS')
iac_pool_pls.groupby('Area')['Mean'].median()